In [3]:
import pandas as pd
import re

In [4]:
import mysql.connector
cnx = mysql.connector.connect(user='vibudh', password='vibudh3399',
                              host='os25.neb-one.gc.ca',
                              database='pcmr')
df = pd.read_sql('SELECT * FROM issues', con=cnx)
cnx.close()
df_loc = df[['tableId', 'rowIndex','content','loc_coord', 'loc_kp', 'loc_mer', 'loc_other', 'loc_tract', 'loc_utm']]
df_loc.head()

,tableId,rowIndex,content,loc_coord,loc_kp,loc_mer,loc_other,loc_tract,loc_utm
0,0114daa6-c048-4081-b4cf-7a9d7461fa44,1,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+020,None,None,None,None
1,0114daa6-c048-4081-b4cf-7a9d7461fa44,2,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+120,None,None,None,None
2,0114daa6-c048-4081-b4cf-7a9d7461fa44,3,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+120,None,None,None,None
3,0114daa6-c048-4081-b4cf-7a9d7461fa44,4,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+220,None,None,None,None
4,0114daa6-c048-4081-b4cf-7a9d7461fa44,5,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+200 to KP 0+513,None,None,None,None


In [12]:
#UTM Locations 
for index, row in df_loc.iterrows():
    raw_text = str(row['loc_utm'])
    if len(raw_text) > 10 and raw_text != 'None':
        print(raw_text)

344948 E 5933706 N to 344946 E 5933656 N
345063 E 5931107 N to 345059 E 5930957 N
345046 E 5930587 N, 344894 E 5930204 N
345046 E 5930587 N, 344894 E 5930204 N
345154 E 5927949 N to 345151 E 5927879 N
345130 E  5927121 N
345099 E  5926312 N
346447 E  5924174 N
350006 E 5924001 N
351076 E 5922176 N
351087 E 5922051 N
351951 E 5920634 N to  351967 E  5920608 N
352052 E 5920531 N
352173 E 5920249 N
352784 E 5919396 N
353195 E 5918636 N
354011 E 5917323 N
354265 E 5916904 N
354369 E 5916769 N
355451 E 5915075 N
355762 E 5914451 N
355945 E 5914176 N
356800 E 5912899 N
357138 E 5912567 N
358102 E 5910790 N
358470 E 5908615 N to 358469 E 5908585 N
358464 E 5908415 N
359851 E 5906263 N
360084 E 5905948 N to 360302 E 5905689 N
361188 E 5904531 N
361206 E 5904508 N to 361218 E 5904492 N
361448 E 5904204 N, 361626 E 5903983 N
361648 E 5903885 N to 361698 E 5903884 N
362256 E 5903618 N to 362363 E 5903544 N
362404 E 5903515 N
363329 E 5902919 N
364064 E 5902357 N
368046 E 5899554 N
372079 E 598682

In [3]:
len(df_loc['loc_coord'])
# for index, row in df_loc.iterrows():
#     mer = str(row['loc_mer'])
#     a = mer[1]
#     if mer != None:
#         print(mer, mer[1], a)
        
# df_loc['loc_mer'][15]

5190

In [4]:
for loc in df_loc['loc_coord']:
    raw_text = str(row['loc_coord'])
    if loc != None:
        1+1
        #print(loc)

In [5]:
raw_text = 'SW 9-52-23/ NW 4-52-23 NE 10-95-7 W4 to NW 15-97-7 W4 10-44-12/ SE 10-44-12'
DLS_REGEX3 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?(W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?|\s*)'
dls_tags = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
print(dls_tags)


# Pattern pattern = Pattern.compile(DLS_REGEX3);
# Matcher matcher = pattern.matcher(dls_tags);
# // Check all occurrences
# System.out.print("Start index: " + matcher.start());
# System.out.print(" End index: " + matcher.end());
# System.out.println(" Found: " + matcher.group());
loc_no_local = 0
for dls_tag in dls_tags:
    meridian = 0
    legal_subdivision, section, township, range_ = dls_tag[0], int(dls_tag[1]), int(dls_tag[3]), int(dls_tag[4])
    loc_no_local = loc_no_local +1
    if dls_tag[6] == '':
        meridian = 1
        
    #print(legal_subdivision)
    string_dls = '{}-{}-{}-{}-W{}M'.format(legal_subdivision, section, township, range_, meridian)
    #print(string_dls, legal_subdivision)
    i = 0
    for m in re.finditer(DLS_REGEX3, raw_text):
        i = i+1
        if i == loc_no_local:
            print(1, raw_text[m.end()-1:m.end()+5])
            
            if ('to' in raw_text[m.end()-1:m.end()+5]) or ':' in raw_text[m.end()-1:m.end()+5]:
                print(2)
                dls_tags_temp = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text[m.end()-1:])))
                if len(dls_tags_temp) > 1:
                    print(dls_tags_temp)



[('SW', '9', '-', '52', '23', '', ''), ('NW', '4', '-', '52', '23', '', ''), ('NE', '10', '-', '95', '7', 'W4', '4'), ('NW', '15', '-', '97', '7', 'W4', '4'), ('', '10', '-', '44', '12', '', ''), ('SE', '10', '-', '44', '12', '', '')]
1  NW 4-
1  NE 10
1 4 to N
2
[('NW', '15', '-', '97', '7', 'W4', '4'), ('', '10', '-', '44', '12', '', ''), ('SE', '10', '-', '44', '12', '', '')]
1 4 10-4
1  SE 10
1 2


In [22]:
import re
LAT_LONG_DD_1_REGEX = r'(\d{1,3}(?:\.\d+))\s*([NWSE](?=[^a-z0-9]|$))'
LAT_LONG_DD_2_REGEX = r'(?i)(lat|long|latitude|longitude)[:\s]+(-?\d{1,3}(?:\.\d+))'
LAT_LONG_DMS_REGEX = r'(\d{1,3}(?:\.\d+)?)[°](\d+(?:\.\d+)?)[\'’](\d+(?:\.\d+)?)[\'\"\’\”]+([NWSE])'
#DLS_REGEX1 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?'
DLS_REGEX3 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?(W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?|\s*)'


#DLS_REGEX2 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?'
#DLS_REGEX3 =  r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?(W[^a-zA-Z0-9]?(?P<meridian>([0-9P])M?)|\s*)'
#r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?((W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?)|\s*)'
NTS_REGEX = r'([a-dA-D])(?P<delim>[^a-zA-Z0-9])(\d{1,3})(?P=delim)([a-lA-L])[^a-zA-Z0-9](\d{1,3})(?P=delim)([a-pA-P])(?P=delim)(1[1-6]|0?[1-9])'
NTS_REGEX2 = r'([a-dA-D])(?P<delim>[^a-zA-Z0-9]|\w[ ]{2,}\w)(\d{1,3})(?P=delim)([a-lA-L])(?P<delim2>[^a-zA-Z0-9])(\d{1,3})(?P=delim)([a-pA-P])(?P=delim)(1[1-6]|0?[1-9])'
MLV_REGEX = 'MLV\)?(\d+)(?:\+(\d+(?:\.\d+)?)(km|m)?)?'

table_id = []
row_id = []
loc_no = []
loc_text = []
loc_format = []
start_loc = []
end_loc = []
# start_
# start_lat = []
# start_long = []
# end_lat = []
# end_long = []
check_raw_text = []

end_point_loc =0

locations_found = 0
rows_with_locs = 0

count_nts = 0
count_lat_lang = 0
count_DMS = 0
count_others = 0
for index, row in df_loc.iterrows():
    
    check = 0
    
    NTS_REGEX2 = r'([a-dA-D])(?P<delim>[^a-zA-Z0-9]|\w[ ]{2,}\w)(\d{1,3})(?P=delim)([a-lA-L])(?P<delim2>[^a-zA-Z0-9])(\d{1,3})(?P=delim)([a-pA-P])(?P=delim)(1[1-6]|0?[1-9])'

    raw_text = str(row['loc_coord'])
    dls_tags = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
    nts_tags = re.findall(NTS_REGEX2, re.sub('\s+', '', ' '.join(raw_text)))
    lat_long_1tags = re.findall(LAT_LONG_DD_1_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
    lat_long_2tags = re.findall(LAT_LONG_DD_2_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
    lat_long_DMStags = re.findall(LAT_LONG_DMS_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
    
    
    loc_no_local = 0
    continue_flag = 0
    #locations_to_insert = len(dls_tags)
    if len(nts_tags) > 1:
        print(raw_text, " NTS " ,nts_tags)
        count_nts += 1 
        
    if len(lat_long_1tags)> 1:
        print(raw_text, " Lat_Lang 1 ", lat_long_1tags)
        count_lat_lang += 1
        
    if len(lat_long_2tags)> 1:
        print(raw_text, " Lat_Lang 2", lat_long_2tags)
        count_lat_lang += 1
        
    if len(lat_long_DMStags)> 1:
        print(raw_text, " Lat_Lang DMS", lat_long_DMStags)
        count_DMS += 1
        
    if len(dls_tags) ==0 and len(nts_tags) ==0 and len(lat_long_1tags)== 0 and len(lat_long_2tags) == 0 and len(lat_long_DMStags) == 0 and len(raw_text) > 0 and raw_text != 'None':
        print(raw_text)
        count_others += 1

    for dls_loc in dls_tags:
        
        
        loc_no_local = loc_no_local + 1
    
        if continue_flag == 1:
            continue_flag = 0
            continue
        
        table_id.append(row['tableId'])
        row_id.append(row['rowIndex'])
        
        loc_no.append(loc_no_local)
        
        loc_text.append(row["loc_coord"])
        loc_format.append("DLS")
        
        legal_subdivision, section, township, range_ = dls_loc[0], int(dls_loc[1]), int(dls_loc[3]), int(dls_loc[4])
        meridian = 0
        if dls_loc[6] == '':
            mer = str(row['loc_mer'])
            meridian = mer[1]
        else:
            meridian = dls_loc[6]
        string_dls = '{}-{}-{}-{}-W{}M'.format(legal_subdivision, section, township, range_, meridian)
        start_loc.append(string_dls)
        
        if meridian == 0 or len(legal_subdivision) !=2:
            check = 1
        
        i = 0
        for m in re.finditer(DLS_REGEX3, raw_text):
            i = i+1
            if i == loc_no_local:
                
                if ('to' in raw_text[m.end()-1:m.end()+5]) or (':' in raw_text[m.end()-1:m.end()+5]):
#                     print(2)
#                     print("found  to in", raw_text)
#                     print("extract loc in", (raw_text[m.end()-1:]))
                    dls_tags_temp = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text[m.end()-1:])))
                    # print(dls_tags_temp)
                    if len(dls_tags_temp) > 0:
                        # print("extracting locations")
                        
                        end_point_loc = end_point_loc +1
                        continue_flag = 1
                        dls_tag_temp = dls_tags_temp[0]
                        
                        legal_subdivision, section, township, range_ = dls_tag_temp[0], int(dls_tag_temp[1]), int(dls_tag_temp[3]), int(dls_tag_temp[4])
                        
                        meridian = 0
                        if dls_tag_temp[6] == '':
                            mer = str(row['loc_mer'])
                            meridian = mer[1]
                        else:
                            meridian = dls_tag_temp[6]
                        string_dls = '{}-{}-{}-{}-W{}M'.format(legal_subdivision, section, township, range_, meridian)
                        end_loc.append(string_dls)
                        
                        if meridian == 0 or len(legal_subdivision) != 2:
                            check = 1
        
        if continue_flag == 0:
            end_loc.append(None)
            
        if check ==0:
            check_raw_text.append(0)
        else:
            check_raw_text.append(1)
        
        



94P-4-J31
94P-4-I40
94P-4-I50
94P-4-I50
94P-4-I49
94P-4-I48
a‑59‑A/94O14 to c‑49‑A/94O14(KP 0.00 to KP 1.00)
c‑49‑A/94O14 to a‑40‑A/94O14(KP 1.00 to KP 2.00)
a‑90‑I/94O11 to a‑80‑I/94O11(KP 7.00 to KP 8.00)
d-50-I/94O11 to a‑49‑I/94O11(KP10.00 to KP 11.00)
a‑49‑I/94O11 to a‑39‑I/94O11(KP 11.00 to KP 12.00)
a‑49‑I/94O11 to a‑39‑I/94O11(KP 11.00 to KP 12.00)
a‑39‑I/94O11 to a‑29‑I/94O11 (KP 12.00 to KP 13.00)
a‑39‑I/94O11 to a‑29‑I/94O11 (KP 12.00 to KP 13.00)
c‑8‑I/94O11 to a‑98‑H/94O11 (KP 14.00 to KP 15.00)
c‑8‑I/94O11 to a‑98‑H/94O11 (KP 14.00 to KP 15.00)
c‑17‑H/94O11 to d‑68‑H/94O11 (KP 17.00 to KP 18.00)
c‑17‑H/94O11 to d‑68‑H/94O11 (KP 17.00 to KP 18.00)
d‑68‑H/94O11 to c‑58‑H/94O11 (KP 18.00 to KP 19.00)
c‑58‑H/94O11 to d‑49‑H/94O11 (KP 19.00 to KP 20.00)
d‑20‑H/94O11 to c‑10‑H/94O11 (KP 23.00 to KP 24.00)
d‑20‑H/94O11 to c‑10‑H/94O11 (KP 23.00 to KP 24.00)
c‑10‑H/94O11 to a‑91‑B/94O11 (KP 24.00 to KP 25.00)
b‑81‑B/94O11 to a‑72‑B/94O11 (KP 226.00 to KP 27.00)
a‑72‑B/94O11 to a‑

In [23]:
print(len(table_id), len(row_id), len(loc_no), len(loc_text), 
      len(loc_format), len(start_loc), len(end_loc), 
      len(check_raw_text), end_point_loc)
print("No of NTS = ", count_nts)
print("No of Lat_Long = ", count_lat_lang)
print("No of DMS = ", count_DMS)
print("Count of Other locations = ", count_others)

3643 3643 3643 3643 3643 3643 3643 3643 33
No of NTS =  18
No of Lat_Long =  0
No of DMS =  0
Count of Other locations =  247


In [7]:
df_locations_extracted = pd.DataFrame({'tableId': table_id, 
                                      'rowIndex': row_id, 
                                      'locNo' : loc_no, 
                                      'locText': loc_text,
                                      'locFormat': loc_format,
                                      'startLoc': start_loc,
                                      'endLoc' : end_loc,
                                      'checkRawText': check_raw_text})
                                       
df_locations_extracted.to_csv('pcmr_locations_dls.csv')

In [164]:

for loc in df_loc['loc_coord']:
    if loc != None:
        try:
            legal_subdivision = int(tag[0])
            # remove the first digit from the legal subdivision until it
            # makes sense. We do this because sometimes the regex includes
            # too many digits from a previous line.
            legal_subdivision = util.truncate_to_abs_value(legal_subdivision, 16)
        except ValueError:
            if tag[0] != '':
                legal_subdivision = tag[0]
            else:
                legal_subdivision = None

        section, township, range_ = int(tag[1]), int(tag[3]), int(tag[4])

        # the meridian can be an integer or 'P'.
        try:
            meridian = int(tag[5])
        except ValueError:
            meridian = tag[5]

        # get string representation of the DLS location
        if legal_subdivision is None:
            string = '{}-{}-{}-W{}M'.format(section, township, range_, meridian)
        else:
            string = '{}-{}-{}-{}-W{}M'.format(legal_subdivision, section, township, range_, meridian)
            
        print(string)

        # convert DLS location to a latitude-longitude coordinate
        lat_long_representation = gis_converter.dls_to_lat_long(legal_subdivision, section, township, range_, meridian)

        dls_tags_result.append(dls_location(
            legal_subdivision=legal_subdivision,
            section=section,
            township=township,
            range=range_,
            meridian=meridian,
            text=string,
            lat_long=lat_long_representation
        ))

NameError: name 'tag' is not defined